# Holiday and Fraud analysis

In [17]:
import sys
sys.path.append("/home/ruitwang/generic-buy-now-pay-later-project-group-3/scripts")
import datetime
import re
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, FloatType
from pyspark.sql import SparkSession, functions as F


In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data_Explorer")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/06 01:03:59 WARN Utils: Your hostname, Loky-PC resolves to a loopback address: 127.0.1.1; using 192.168.55.225 instead (on interface eth0)
22/10/06 01:03:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 01:04:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [60]:
import pandas as pd
data = ['2021-03-08','2021-04-02','2021-04-03','2021-04-04','2021-04-05','2021-04-25','2021-06-14','2021-09-24','2021-11-02','2021-12-25','2021-12-26','2021-12-27','2021-12-28',
       '2022-01-01','2022-01-03','2022-01-26','2022-03-14','2022-04-15','2022-04-16','2022-04-17','2022-04-18','2022-04-25',
       '2022-06-13','2022-09-22','2022-09-23']
dates = pd.date_range(start='2021-02-01', end='2022-10-31')

list1 = []
for date in dates:
    if date in data:
        list1.append((date.strftime('%Y-%m-%d'), True))
    else:
        list1.append((date.strftime('%Y-%m-%d'), False))
holiday_data = spark.createDataFrame(pd.DataFrame(list1, columns =['Date', 'holiday?']))
holiday_data = holiday_data.withColumn("Date",col("Date").cast(DateType()))



In [93]:
from operator import index

from numpy import inner


holiday_pd = holiday_data.toPandas()

consumer_fraud = pd.read_csv("../data/tables/consumer_fraud_probability.csv")
consumer_fraud["date"] = pd.to_datetime(consumer_fraud["order_datetime"])
consumer_fraud.drop(["order_datetime"])
fraud_by_date = consumer_fraud.groupby("order_datetime").mean("fraud_probablity")
holiday_pd["date"] = pd.to_datetime(holiday_pd["Date"])

fraud_holiday = holiday_pd.join(fraud_by_date, how = 'right')
fraud_holiday.head()




KeyError: "['order_datetime'] not found in axis"